In [ ]:
# imports

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx

In [ ]:
# data loading

raw_data = np.load('data/calms21_task3_train.npy', allow_pickle=True)
raw_features = raw_data[()]['approach']['task3/approach/train/mouse001_task3_approach']['keypoints']
labels = raw_data[()]['approach']['task3/approach/train/mouse001_task3_approach']['annotations']

In [ ]:
# data reshaping

features = np.swapaxes(raw_features, 2,3).reshape(-1,14,2)

# tmp = []
# for frame in raw_features:
#     mouse1 = zip(frame[0][0].tolist(), frame[0][1].tolist())
#     mouse2 = zip(frame[1][0].tolist(), frame[1][1].tolist())
#     # print(list(mouse1))
#     tmp.append(list(mouse1) + list(mouse2))
# features = tmp

In [ ]:
# putting the data in a graph

mouse1 = [0,1,0,2,1,3,1,2,3,4,3,5,4,6,5,6]
mouse2 = [x+7 for x in mouse1]
first_option = mouse1+[3,10]+mouse2

i = [0,0,1,1,3,3,4,5]
j = [1,2,3,2,4,5,6,6]
i2 = [x+7 for x in i]
j2 = [x+7 for x in j]
second_option = [i+[3]+i2,j+[10]+j2]

edge_index = torch.tensor(second_option, dtype=torch.long)
x = torch.tensor(features[0], dtype=torch.float)

graph = Data(x=x, edge_index=edge_index)

In [ ]:
# visualizing the graph

print(graph.edge_index.t().tolist())

G = to_networkx(graph, to_undirected=True)

# method 1
G = nx.DiGraph(G)
nx.draw(G)

# method 2
def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()
    
visualize(G, color=graph.y)